# Tutorial CV1 - Photon redirection

This tutorial runs Penny Lane's "hello world" example of continuous-variable quantum computation with the Strawberry Fields plugin. Starting with a photon in Mode 1, the goal is to optimize a beam splitter to redirect the photon to Mode 2. 

We compare the gradient descent optimizer without and with momentum for this task.

### Imports

First we need to import Penny Lane. This allows us to automatically compute gradients for functions that manipulate numpy arrays, including quantum functions.

In [9]:
import pennylane as qml
import numpy as np
from pennylane.optimize import GradientDescentOptimizer, MomentumOptimizer

ModuleNotFoundError: No module named 'pennylane_sf'

Next, create a "device" to run the quantum node. We use StrawberryFields to simulate a photonic quantum processor with two quantum modes.

In [8]:
try:
    dev = qml.device('strawberryfields.fock', wires=2, cutoff_dim=10)
    
except:
    print("To run this tutorial you need to install the strawberryfields plugin...")

To run this tutorial you need to install the strawberryfields plugin...


### Quantum node

We define a quantum circuit which starts with one photon into the first mode and sends both modes through a beam splitter. 
 
*Note: The `qml.qnode(dev)` decorator does the same as creating a qnode via `circuit = qml.qnode.QNode(circuit, dev)`.*

In [ ]:
@qml.qnode(dev)
def circuit(var):

    qml.FockState(1, [0])
    qml.Beamsplitter(var[0], var[1], [0, 1])

    return qml.expval.MeanPhoton(0)

This function uses Penny Lane to run the following quantum circuit:

<img src="figures/redirection_circuit.png">

Depending on the variable $v_1$, the transmission parameter of the beam splitter, the photon remains in the first mode (for $v_1 = n \pi$ with integer $n$), gets redirected to the second mode (for $v_2 = n\pi / 2$) or is in a superposition of being in both modes. The phase parameter $v_2$ does not have any influence on the result, so we expect it to have a zero gradient and not to get updated at all.

### Objective

Next, we define a cost that quantifies whether the redirection is successful. Since we want to maximize the expectation of the photon number in the second mode, we can minize expectation of the photon number in the first mode. (Alternatively, we could also minimize the negative of the photon number in the second mode.)

In [ ]:
def objective(var):
    return circuit(var)

This objective has the following optimization landscape. *Note: To run the following cell you need the matplotlib library.*

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline  
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import MaxNLocator

fig = plt.figure(figsize = (6, 4))
ax = fig.gca(projection='3d')

X = np.arange(-3.1, 1.6, 0.2)
Y = np.arange(-1., 1.25, 0.25)

xx, yy = np.meshgrid(X, Y)
Z = np.array([[objective([x, y]) for x in X] for y in Y]).reshape(len(Y), len(X))
surf = ax.plot_surface(xx, yy, Z, cmap=cm.coolwarm, antialiased=False)

ax.set_xlabel("v1")
ax.set_ylabel("v2")
ax.zaxis.set_major_locator(MaxNLocator(nbins = 5, prune = 'lower'))
ax.yaxis.set_major_locator(MaxNLocator(nbins = 4, prune = 'lower'))

plt.show()

As stated above, the gradient does not depend on the phase parameter $v_2$. 

### Optimization

We choose a simple Gradient Descent Optimizer with a step size of 0.1.

In [ ]:
gd = GradientDescentOptimizer(stepsize=0.1)

The initial values of the variabels are set to near-zero. This corresponds to an identity gate, in other words, the circuit leaves the photon in the first mode. *Note that at zero exactly the gradient vanishes and the optimization algorithm will not descent from the maximum.*

In [ ]:
var_init = np.array([0.01, -0.5])

We optimize the weights for 100 steps.

In [ ]:
var = var_init
var_gd = []

for iteration in range(100):
    var = gd.step(objective, var)
    var_gd.append(var)
    
    if iteration % 10 == 0:
        print('Cost after step {:3d}: {:0.7f} | Variables [{:0.7f}, {:0.7f}]'
              ''.format(iteration, objective(var), var[0], var[1]))

Starting at a different offset, we train the MomentumOptimizer, which improves on gradient descent by making an update dependent on the previous gradient.

In [ ]:
mm = MomentumOptimizer(stepsize=0.1, momentum=0.5)

var_init = np.array([0.01, 0.5])

var = var_init
var_mm = []

for iteration in range(100):
    var = mm.step(objective, var)
    var_mm.append(var)

    if iteration % 10 == 0:
        print('Cost after step {:3d}: {:0.7f} | Variables [{:0.7f}, {:0.7f}]'
              ''.format(iteration, objective(var), var[0], var[1]))

In [ ]:
fig = plt.figure(figsize = (6, 4))
ax = fig.gca(projection='3d')

X = np.arange(-3.1, 1.6, 0.2)
Y = np.arange(-1., 1.25, 0.25)

xx, yy = np.meshgrid(X, Y)
Z = np.array([[objective([x, y]) for x in X] for y in Y]).reshape(len(Y), len(X))
surf = ax.plot_surface(xx, yy, Z, cmap=cm.coolwarm, antialiased=False)

path_z = [objective(var)+1e-8 for var in var_gd]
path_x = [var[0] for var in var_gd]
path_y = [var[1] for var in var_gd]
ax.plot(path_x, path_y, path_z, c='green', marker='.', label="graddesc")

path_z = [objective(var)+1e-8 for var in var_mm]
path_x = [var[0] for var in var_mm]
path_y = [var[1] for var in var_mm]
ax.plot(path_x, path_y, path_z, c='purple', marker='.', label="momentum")

ax.set_xlabel("v1")
ax.set_ylabel("v2")
ax.zaxis.set_major_locator(MaxNLocator(nbins = 5, prune = 'lower'))
ax.yaxis.set_major_locator(MaxNLocator(nbins = 4, prune = 'lower'))

plt.legend()
plt.show()